# CSCI 184 Machine Learning Final Project: Inflation Detector
# Group Members: Arjun Chimni, Patrick Callahan, Andrew Schulz


## Project Idea: 	
  >With the recent CPI numbers coming out and a recession on the loom, we wanted to identify what conditions cause inflation before the inflation happens. We think that this topic is especially pertinent at the current moment. We want to identify the features/conditions under which inflation happens and try to predict if there will be inflation to the money supply. 
  
  >To do this, we will analyze employment data, CPI (consumer price index), PPI (producer price index), and confidence indexes in order to obtain an idea about what causes inflation and how to detect it early. We will train historical data of all these features to draw conclusions.
  
  > Ideally, the program will be able to suggest some sort of action in order to combat inflation or just general advice for the average consumer heading into a high inflation landscape.

